In [0]:
import os
import subprocess
import os
import zipfile
import cv2
import numpy as np
import os
import zipfile
from moviepy.video.io.VideoFileClip import VideoFileClip
from google.colab.patches import cv2_imshow

In [0]:
less_35=[]
file_no_face=[]
detector = cv2.dnn.readNetFromCaffe('/cascade/deploy.prototxt.txt',
                                    '/cascade/res10_300x300_ssd_iter_140000.caffemodel')

entries=['Anger','Disgust','Fear','Happy','Neutral','Sad','Surprise']

if __name__ == '__main__' :
    for entry in entries: 
        entries2=os.listdir('/CAER/train/'+entry)
        nice=[]
        cc=0

        for item in entries2:        
            cap = cv2.VideoCapture('/CAER/train/'+str(entry)+'/'+item);
            clip = VideoFileClip("/CAER/train/"+str(entry)+'/'+item)
                    
            file_name=(item.split('.avi'))[0]

            ## Setting the frame limit to 100
            cap.set(cv2.CAP_PROP_FRAME_COUNT, 400)
            length=50
            count=0
            name_count=1
            print(file_name+" start")
            while(cap.isOpened()):
                count+=1
                if length ==count:
                    break
                if name_count==6:
                    break
                ret, img = cap.read()
                if img is None:
                    continue
                inputBlob = cv2.dnn.blobFromImage(cv2.resize(img, (224, 224)), 1, (300, 300), (104, 177, 123))
                detector.setInput(inputBlob)
                detections = detector.forward()
                for i in range(0, detections.shape[2]):
                    # Probability of prediction
                    prediction_score = detections[0, 0, i, 2]
                    if prediction_score < 0.6:
                        continue
                    # Finding height and width of frame
                    (h, w) = img.shape[:2]
                    # compute the (x, y)-coordinates of the bounding box for the
                    # object
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (x1, y1, x2, y2) = box.astype("int")
                    
                    if y2 > h or x2 > w:
                        continue
                    if y2 <= y1 or x2 <= x1:
                        continue
                    if y2 ==0 or x2 ==0 or y1 ==0 or x1 ==0:
                        continue
                    if y2 -y1<=0 or x2 - x1<=0:
                        continue
                    if y1 < 0 or y2<0 or x1<0 or x2<0:
                        continue
                    img2 = img[y1:y2, x1:x2]
                    face_name = '/data/imageData/training_face/'+str(entry)+'/'+str(file_name)+'_' + str(name_count) + '.jpg'
                    cv2.imwrite(face_name, img2)
                    blank_image = np.zeros(shape=[y2-y1, x2-x1, 3], dtype=np.uint8)
                    img[y1:y2, x1:x2] = blank_image
                    background_name = '/data/imageData/training_background/'+str(entry)+'/'+str(file_name)+'_' + str(name_count) + '.jpg'
                    name_count+=1
                    cv2.imwrite(background_name, img)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
            if(name_count!=6):
                print(item)
                less_35.append(item+":"+str(entry))
            else:
                print("Done:"+item +', ' + entry + " " + str(cc))
                nice.append(item)
                cc+=1

        print('complete'+str(entry))

In [0]:
def deleteDataset(entry,nice=[]):
    path = '/CAER/train/'+ entry
    entries=os.listdir(path)
    
    for item in entries:
        if (item not in nice):
            os.remove('/CAER/train/'+ entry + '/' + item)

deleteDataset(entry,nice)
